In [ ]:
import polars as pd
import plotly.express as px
import pw_mapping

In [ ]:
pages = pw_mapping.Page.select()
df_pages = pd.DataFrame([{"page_id": page.page_id, "title": page.title, "item_id": page.item_id, "views": page.views} for page in pages])
df_pages

In [ ]:
t_df = df_pages.filter(df_pages["views"] < 200)
px.histogram(t_df, x="views", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql("select item_id, count(item_id) from item_aliases group by item_id")
df_aliases = pd.DataFrame([{"item_id": alias[0], "count": alias[1]} for alias in cursor])

In [ ]:
t_df = df_aliases.filter(df_aliases["count"] < 80)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql("select source_item_id, count(source_item_id) from statements group by source_item_id limit 1000000")
df_sources = pd.DataFrame([{"item_id": source[0], "count": source[1]} for source in cursor])

In [ ]:
t_df = df_sources.filter(df_sources["count"] < 250)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql(
    "select target_item_id, count(target_item_id) from statements group by target_item_id limit 1000000")
df_targets = pd.DataFrame([{"item_id": target[0], "count": target[1]} for target in cursor])

In [ ]:
t_df = df_targets.filter(df_targets["count"] < 250)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql(
    "select edge_property_id, count(edge_property_id) from statements group by edge_property_id limit 1000000")
df_props = pd.DataFrame([{"item_id": prop[0], "count": prop[1]} for prop in cursor])

In [ ]:
t_df = df_props.filter(df_props["count"] < 10000)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
df_joined = df_pages.join(df_aliases, on="item_id", how="left")
df_joined = df_joined.drop_nulls()
df_joined

In [ ]:
t_df = df_joined.filter(df_joined["views"] < 400)
px.scatter(t_df, x="views", y="count")